<a href="https://colab.research.google.com/github/calistasalscpw/Selena-Finance-Tracker/blob/risma/Cleaning_NewData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DATA ANALYSIS**

In [1]:
import numpy as np
import pandas as pd

In [2]:
pt_df = pd.read_csv("personal_transactions.csv")
pt_df.sample(20)

,Date,Description,Amount,Transaction Type,Category,Account Name
706,07/17/2019,City Water Charges,35.00,debit,Utilities,Checking
96,03/23/2018,Credit Card Payment,559.91,credit,Credit Card Payment,Platinum Card
417,11/19/2018,Hardware Store,10.02,debit,Home Improvement,Silver Card
175,05/12/2018,Fancy Restaurant,78.00,debit,Restaurants,Platinum Card
634,05/14/2019,Grocery Store,2.02,debit,Groceries,Silver Card
522,02/11/2019,BP,24.01,debit,Gas & Fuel,Platinum Card
354,10/02/2018,Credit Card Payment,124.03,credit,Credit Card Payment,Platinum Card
750,08/17/2019,Barbershop,19.00,debit,Haircut,Platinum Card
388,11/01/2018,Amazon,13.13,debit,Shopping,Platinum Card
110,04/01/2018,Amazon,13.13,debit,Shopping,Platinum Card


In [3]:
def assessing_data(df):
    print(f"\n---TABLE INFO---\n")
    df.info()

    print(f"\n---MISSING VALUES---\n")
    print(df.isna().sum())

    print(f"\n---DUPLICATE DATA---\n")
    print(df.duplicated().sum())

    print(f"\n---STATISTICS PARAMETER---\n")
    print(df.describe())

In [4]:
assessing_data(pt_df)


---TABLE INFO---

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              806 non-null    object 
 1   Description       806 non-null    object 
 2   Amount            806 non-null    float64
 3   Transaction Type  806 non-null    object 
 4   Category          806 non-null    object 
 5   Account Name      806 non-null    object 
dtypes: float64(1), object(5)
memory usage: 37.9+ KB

---MISSING VALUES---

Date                0
Description         0
Amount              0
Transaction Type    0
Category            0
Account Name        0
dtype: int64

---DUPLICATE DATA---

0

---STATISTICS PARAMETER---

            Amount
count   806.000000
mean    273.391489
std     667.630374
min       1.750000
25%      15.687500
50%      37.480000
75%     117.680000
max    9200.000000


### **CLEANING FOR DATASET**

In [5]:
def preprocess_data_from_kaggle(data):

  usd_to_idr = 15865
  data['Amount'] = data['Amount'] * usd_to_idr
  data.rename(columns={'Amount':'amount'}, inplace=True)

  data.rename(columns={'Date':'date'}, inplace=True)
  data['date'] = pd.to_datetime(data['date']).dt.date

  data['Transaction Type'] = data['Transaction Type'].replace({'debit': 'Expense', 'credit': 'Income'})
  data.rename(columns={'Transaction Type':'transaction_type'}, inplace=True)
  data['transaction_type'] = data['transaction_type'].astype('category')

  data.drop(["Description","Category","Account Name"],
            axis = 1, inplace= True)

  data.dropna(inplace=True)

  data.sort_values(by='date', ascending=True)

  return data

In [6]:
preprocess_data_from_kaggle(pt_df)

,date,amount,transaction_type
0,2018-01-01,176260.15,Expense
1,2018-01-02,19790635.60,Expense
2,2018-01-02,384250.30,Expense
3,2018-01-03,36459197.85,Income
4,2018-01-04,186572.40,Expense
...,...,...,...
801,2019-09-27,35696250.00,Income
802,2019-09-28,530842.90,Expense
803,2019-09-28,67743.55,Expense
804,2019-09-30,27763.75,Expense


### **CLEANING DATA FOR CASHFLOW ANALYSIS AND ANOMALY**

In [7]:
def preprocess_clean_data(data):

    data['income'] = data.apply(lambda x: x['amount'] if x['transaction_type'] == 'Income' else 0, axis=1)
    data['expense'] = data.apply(lambda x: x['amount'] if x['transaction_type'] == 'Expense' else 0, axis=1)

    income_df = data[data['transaction_type'] == 'Income'][['date', 'amount']].copy()
    expense_df = data[data['transaction_type'] == 'Expense'][['date', 'amount']].copy()

    income_df = income_df[income_df['amount'] > 0]
    expense_df = expense_df[expense_df['amount'] > 0]

    return income_df, expense_df

In [8]:
income_df, expense_df = preprocess_clean_data(pt_df)

In [9]:
income_df.head()

,date,amount
3,2018-01-03,36459197.85
13,2018-01-12,31730000.00
20,2018-01-19,31730000.00
22,2018-01-22,8804916.35
23,2018-01-22,4915135.65


In [10]:
expense_df.head()

,date,amount
0,2018-01-01,176260.15
1,2018-01-02,19790635.60
2,2018-01-02,384250.30
4,2018-01-04,186572.40
5,2018-01-05,410110.25


In [11]:
with open('pemasukan_new.csv', 'w') as csv_file:
    income_df['date'] = pd.to_datetime(income_df['date'])
    income_df['date'] = income_df['date'].dt.strftime('%Y-%m-%d')
    income_df.to_csv(csv_file, index=False)

with open('pengeluaran_new.csv', 'w') as csv_file:
    expense_df['date'] = pd.to_datetime(expense_df['date'])
    expense_df['date'] = expense_df['date'].dt.strftime('%Y-%m-%d')
    expense_df.to_csv(csv_file, index=False)

In [12]:
import json

income_df = pd.read_csv("pemasukan_new.csv")
income_df.to_json('pemasukan_new.json', orient='records', indent=1)

expense_df = pd.read_csv("pengeluaran_new.csv")
expense_df.to_json('pengeluaran_new.json', orient='records', indent=1)